In [1]:
import Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()
using LinearAlgebra, Plots
import ForwardDiff as FD
# import MeshCat as mc 
# using JLD2
using Test
# using Random
# include(joinpath(@__DIR__,"utils/cartpole_animation.jl"))
# include(joinpath(@__DIR__,"utils/basin_of_attraction.jl"))

  Activating environment at `~/nCBF-drone/danaus_ros_ws/julia_eth/Project.toml`


## TODO

### 1. Verify that Simin's dynamics works, in particular the pendulum dynamics
In order to this, we need to use the ETH dynamics, in particular ETH's pendulum dynamics. We know for a fact that the ETH dynamics works in real life since we successfully deployed.
- Simulation Dynamics: ETH
- Observer: Convert $r$ and $s$ into roll and pitch of pendulum, and use the time-differentiation for the velocities.
- Controller Dynamics: Simin's

- Control inputs: $a, \omega_x, \omega_y, \omega_z$
    - [P0] Ensure that Simin's dynamics has the same coordinate system for both the dynamics and controls!

Repeat 1~3 for $dt = 0.01$:
1. State <-- ETH Dynamics
2. New State <-- Observer
    - Check how much error there is between these two!
3. Control Input <-- Simin's Dynamics

### 2. See if torque-to-body-rate conversion may work in low-fidelity simulation at 100 Hz
We want to see the naive torque-to-body-rate (TTBR) can stabilize the pendulum at 100 Hz. For this, we do the following:
- Simulation Dynamics: ETH for torque
- Observer: Time-differentiation for $r$ and $s$ OR roll and pitch of pendulum.
- Controller Dynamics: 
    1. Simin's if Task #1 proves that her dynamics is good.
    2. ETH's dynamics with torque control if not.

In [2]:
function rk4(dyn::Function, params::NamedTuple, x::Vector, u::Vector, dt::Float64)
    k1 = dt*dyn(params, x, u)
    k2 = dt*dyn(params, x + 0.5*k1, u)
    k3 = dt*dyn(params, x + 0.5*k2, u)
    k4 = dt*dyn(params, x + k3, u)
    
    return x + (1/6)*(k1 + 2*k2 + 2*k3 + k4)
end

rk4 (generic function with 1 method)

In [3]:
function eth_bodyrate_dynamics(params::NamedTuple, X::Vector, u::Vector)
    """
    The dynamics of the ETH quadrotor-pendulum model.
    This dynamics has 13 states and 4 inputs.
    The inputs are body-rates and overall thrust.
    The thrust is NOT normalized about the hovering position.
    """
    g = params.g
    L = params.L

    γ, β, α, x, y, z, x_dot, y_dot, z_dot, r, s, r_dot, s_dot = X
    a, wx, wy, wz = u
    
    Rx = [1 0 0; 0 cos(γ) -sin(γ); 0 sin(γ) cos(γ)]
    Ry = [cos(β) 0 sin(β); 0 1 0; -sin(β) 0 cos(β)]
    Rz = [cos(α) -sin(α) 0; sin(α) cos(α) 0; 0 0 1]
    R = Rz*Ry*Rx

    pos_ddot = R * [0; 0; a] + [0; 0; -g]
    x_ddot, y_ddot, z_ddot = pos_ddot
    
    rot_dot = [cos(β)*cos(γ) -sin(γ) 0; cos(β)*sin(γ) cos(γ) 0; -sin(β) 0 1]\[wx; wy; wz]
    γ_dot, β_dot, α_dot = rot_dot

    ξ = sqrt(L^2 - r^2 - s^2)

    r_ddot = (L^4*x_ddot - r^3*s_dot^2 + r^4*x_ddot + r*s^3*y_ddot + r^3*s*y_ddot + L^2*r*r_dot^2 + L^2*r*s_dot^2 - 2*L^2*r^2*x_ddot - L^2*s^2*x_ddot + g*r^3*(L^2 - r^2 - s^2)^(1/2) - r*r_dot^2*s^2 + r^3*z_ddot*(L^2 - r^2 - s^2)^(1/2) + r^2*s^2*x_ddot - L^2*r*s*y_ddot + 2*r^2*r_dot*s*s_dot - L^2*g*r*(L^2 - r^2 - s^2)^(1/2) - L^2*r*z_ddot*(L^2 - r^2 - s^2)^(1/2) + g*r*s^2*(L^2 - r^2 - s^2)^(1/2) + r*s^2*z_ddot*(L^2 - r^2 - s^2)^(1/2))/(L^2*(- L^2 + r^2 + s^2))
    s_ddot = (L^4*y_ddot - r_dot^2*s^3 + s^4*y_ddot + r*s^3*x_ddot + r^3*s*x_ddot + L^2*r_dot^2*s + L^2*s*s_dot^2 - L^2*r^2*y_ddot - 2*L^2*s^2*y_ddot + g*s^3*(L^2 - r^2 - s^2)^(1/2) - r^2*s*s_dot^2 + s^3*z_ddot*(L^2 - r^2 - s^2)^(1/2) + r^2*s^2*y_ddot - L^2*r*s*x_ddot + 2*r*r_dot*s^2*s_dot - L^2*g*s*(L^2 - r^2 - s^2)^(1/2) - L^2*s*z_ddot*(L^2 - r^2 - s^2)^(1/2) + g*r^2*s*(L^2 - r^2 - s^2)^(1/2) + r^2*s*z_ddot*(L^2 - r^2 - s^2)^(1/2))/(- L^4 + L^2*r^2 + L^2*s^2)

    return [γ_dot; β_dot; α_dot; x_dot; y_dot; z_dot; x_ddot; y_ddot; z_ddot; r_dot; s_dot; r_ddot; s_ddot]
end

eth_bodyrate_dynamics (generic function with 1 method)

In [12]:
function our_torque_dynamics(params::NamedTuple, X::Vector, u::Vector)
    """
    The dynamics of our quadrotor-pendulum model.
    This dynamics has 16 states and 4 inputs.
    The inputs are also bodyrate and overall thrust.
    The thrust is NOT normalized about the hovering position.
    """
    J = params.J
    g = params.g
    L = params.L
    if L < 1.0
        display("We assume that the CoM of pendulum is at the center of pendulum.")
    end
    
    display("The CoM of the pendulum is at $L m.")

    γ, β, α, γ_dot, β_dot, α_dot, x, y, z, x_dot, y_dot, z_dot, θ, ϕ, θ_dot, ϕ_dot = X
    a, τx, τy, τz = u
    
    Rx = [1 0 0; 0 cos(γ) -sin(γ); 0 sin(γ) cos(γ)]
    Ry = [cos(β) 0 sin(β); 0 1 0; -sin(β) 0 cos(β)]
    Rz = [cos(α) -sin(α) 0; sin(α) cos(α) 0; 0 0 1]
    R = Rz*Ry*Rx

    pos_ddot = R * [0; 0; a] + [0; 0; -g]
    x_ddot, y_ddot, z_ddot = pos_ddot
    
    τ = [τx; τy; τz]
    
    ####################
    ##### Original #####
    ####################
    ω = [γ_dot; 0; 0] + Rx\[0; β_dot; 0] + Rx\Ry\[0; 0; α_dot]  # ω = [ωx, ωy, ωz]    # same thing as below
    # ω = [1 0 -sin(β); 0 cos(γ) cos(β)*sin(γ); 0 -sin(γ) cos(β)*cos(γ)]*[γ_dot; β_dot; α_dot]    # same thing as above
    # ω = 
    rot_ddot = R * (J \ (τ)) # - cross(ω, J*ω)))
    γ_ddot, β_ddot, α_ddot = rot_ddot
    ####################
    
    # ####################
    # ####### New ######## 
    # ####################
    # A = [1 0 -sin(β); 0 cos(γ) cos(β)*sin(γ); 0 -sin(γ) cos(β)*cos(γ)]
    # ω = A*[γ_dot; β_dot; α_dot]
    # A_dot = [0 0 -cos(β)*β_dot; 0 -sin(γ)*γ_dot -sin(β)*sin(γ)*β_dot + cos(β)*cos(γ)*γ_dot; 0 -cos(γ)*γ_dot -sin(β)*cos(γ)*β_dot - cos(β)*sin(γ)*γ_dot]
    # ω_dot = J \ (τ - cross(ω, J*ω))
    # rot_dot = [γ_dot; β_dot; α_dot]
    # rot_ddot = A \ (ω_dot - A_dot*rot_dot)
    # γ_ddot, β_ddot, α_ddot = rot_ddot
    # ####################
    
    θ_ddot = 3/(2*(2*L)) * (-x_ddot*cos(θ) - y_ddot*sin(ϕ)*sin(θ) + (z_ddot+g)*cos(ϕ)*sin(θ)) - ϕ_dot^2*sin(θ)*cos(θ)
    ϕ_ddot = 3/(2*(2*L)*cos(θ)) * (y_ddot*cos(ϕ) + (z_ddot+g)*sin(ϕ)) + 2*θ_dot*ϕ_dot*tan(θ)

    return [γ_dot; β_dot; α_dot; γ_ddot; β_ddot; α_ddot; x_dot; y_dot; z_dot; x_ddot; y_ddot; z_ddot; θ_dot; ϕ_dot; θ_ddot; ϕ_ddot]
end

our_torque_dynamics (generic function with 1 method)

In [ ]:
function our_torque_aug_dynamics(params::NamedTuple, X::Vector, u::Vector)
    """
    The dynamics of our quadrotor-pendulum model.
    This dynamics has 16 states and 4 inputs.
    The inputs are also bodyrate and overall thrust.
    The thrust is NOT normalized about the hovering position.
    """
    J = params.J
    g = params.g
    L = params.L
    if L < 1.0
        display("We assume that the CoM of pendulum is at the center of pendulum.")
    end
    
    display("The CoM of the pendulum is at $L m.")

    γ, β, α, γ_dot, β_dot, α_dot, ωx, ωy, ωz, x, y, z, x_dot, y_dot, z_dot, θ, ϕ, θ_dot, ϕ_dot = X
    a, τx, τy, τz = u
    
    Rx = [1 0 0; 0 cos(γ) -sin(γ); 0 sin(γ) cos(γ)]
    Ry = [cos(β) 0 sin(β); 0 1 0; -sin(β) 0 cos(β)]
    Rz = [cos(α) -sin(α) 0; sin(α) cos(α) 0; 0 0 1]
    R = Rz*Ry*Rx

    pos_ddot = R * [0; 0; a] + [0; 0; -g]
    x_ddot, y_ddot, z_ddot = pos_ddot
    
    τ = [τx; τy; τz]
    
    ####################
    ##### Original #####
    ####################
    ω = [ωx; ωy; ωz]    # same thing as below
    ω_dot = J \ (τ - cross(ω, J*ω))
    ωx_dot, ωy_dot, ωz_dot = ω_dot
    rot_ddot = R * (J \ (τ - cross(ω, J*ω)))
    γ_ddot, β_ddot, α_ddot = rot_ddot
    ####################
    
    # ####################
    # ####### New ######## 
    # ####################
    # A = [1 0 -sin(β); 0 cos(γ) cos(β)*sin(γ); 0 -sin(γ) cos(β)*cos(γ)]
    # ω = A*[γ_dot; β_dot; α_dot]
    # A_dot = [0 0 -cos(β)*β_dot; 0 -sin(γ)*γ_dot -sin(β)*sin(γ)*β_dot + cos(β)*cos(γ)*γ_dot; 0 -cos(γ)*γ_dot -sin(β)*cos(γ)*β_dot - cos(β)*sin(γ)*γ_dot]
    # ω_dot = J \ (τ - cross(ω, J*ω))
    # rot_dot = [γ_dot; β_dot; α_dot]
    # rot_ddot = A \ (ω_dot - A_dot*rot_dot)
    # γ_ddot, β_ddot, α_ddot = rot_ddot
    # ####################
    
    θ_ddot = 3/(2*(2*L)) * (-x_ddot*cos(θ) - y_ddot*sin(ϕ)*sin(θ) + (z_ddot+g)*cos(ϕ)*sin(θ)) - ϕ_dot^2*sin(θ)*cos(θ)
    ϕ_ddot = 3/(2*(2*L)*cos(θ)) * (y_ddot*cos(ϕ) + (z_ddot+g)*sin(ϕ)) + 2*θ_dot*ϕ_dot*tan(θ)

    return [γ_dot; β_dot; α_dot; γ_ddot; β_ddot; α_ddot; ωx_dot; ωy_dot; ωz_dot; x_dot; y_dot; z_dot; x_ddot; y_ddot; z_ddot; θ_dot; ϕ_dot; θ_ddot; ϕ_ddot]
end

In [39]:
function observer(params::NamedTuple, X_prev::Vector, X::Vector)
    """
    Incoming X has 13 states.
    Outgoing X has 16 states.
    """
    g = params.g
    L = params.L
    dt = params.dt

    # Unpack the states, according to ETH model
    γ_prev, β_prev, α_prev, _, _, _, _, _, _, _, _, _, _ = X_prev
    γ, β, α, x, y, z, x_dot, y_dot, z_dot, r, s, r_dot, s_dot = X

    ζ = sqrt(L^2 - r^2 - s^2)
    θ = atan(r, ζ)
    ϕ = atan(-s, ζ)
    θ_dot = r_dot/L
    ϕ_dot = -s_dot/L

    γ_dot = (γ - γ_prev) / dt
    β_dot = (β - β_prev) / dt
    α_dot = (α - α_prev) / dt

    return [γ; β; α; γ_dot; β_dot; α_dot; x; y; z; x_dot; y_dot; z_dot; θ; ϕ; θ_dot; ϕ_dot]

end

observer (generic function with 1 method)

In [46]:
function torque_to_bodyrate(params::NamedTuple, U_torque::Vector, U_bodyrate_prev::Vector, X::Vector)
    dt = params.dt
    J = params.J
   
    a, τx, τy, τz = U_torque
    a_prev, wx_prev, wy_prev, wz_prev = U_bodyrate_prev
    γ, β, α, γ_dot, β_dot, α_dot, x, y, z, x_dot, y_dot, z_dot, θ, ϕ, θ_dot, ϕ_dot = X

    Rx = [1 0 0; 0 cos(γ) -sin(γ); 0 sin(γ) cos(γ)]
    Ry = [cos(β) 0 sin(β); 0 1 0; -sin(β) 0 cos(β)]
    Rz = [cos(α) -sin(α) 0; sin(α) cos(α) 0; 0 0 1]

    ω = [γ_dot; 0; 0] + Rx\[0; β_dot; 0] + Rx\Ry\[0; 0; α_dot]  # ω = [ωx, ωy, ωz]
    
    w_prev = [wx_prev; wy_prev; wz_prev]
    τ = [τx; τy; τz]
    w_dot = J \ (τ - cross(ω, J*ω))
    w_new = w_prev + w_dot*dt

    return [a; w_new...]
end

torque_to_bodyrate (generic function with 1 method)

In [13]:
#############################################
#################### LQR ####################
#############################################

### Define the parameters ###
nx = 16
nu = 4

dt = 1/100  # 100 Hz

J = [0.00320868 0.00011707 0.00004899; 0.00011707 0.00288707  0.00006456; 0.00004899 0.00006456  0.00495141]
g = 9.81
L = 0.5 # CoM of pendulum from its base
params = (J = J, g = g, L = L, dt = dt)

### Define the goals ###
# γ, β, α, γ_dot, β_dot, α_dot, x, y, z, x_dot, y_dot, z_dot, θ, ϕ, θ_dot, ϕ_dot
xgoal = [0; 0; 0; 0; 0; 0; 0; 0; 3; 0; 0; 0; 0; 0; 0; 0]
# a, τx, τy, τz
ugoal = [g; 0; 0; 0]
# γ, β, α, x, y, z, x_dot, y_dot, z_dot, r, s, r_dot, s_dot
xgoal_eth = [0; 0; 0; 0; 0; 3; 0; 0; 0; 0; 0; 0; 0]
# a, wx, wy, wz
ugoal_eth = [g; 0; 0; 0]

A_ours = FD.jacobian(x -> rk4(our_torque_dynamics, params, x, ugoal, dt), xgoal)
B_ours = FD.jacobian(u -> rk4(our_torque_dynamics, params, xgoal, u, dt), ugoal)
A_eth = FD.jacobian(x -> rk4(eth_bodyrate_dynamics, params, x, ugoal_eth, dt), xgoal_eth)
B_eth = FD.jacobian(u -> rk4(eth_bodyrate_dynamics, params, xgoal_eth, u, dt), ugoal_eth)

################ OURS ################
Q = 1.0*diagm([0; 0; 0; 0; 0; 0; 1; 1; 1; 0; 0; 0; 0.5; 0.5; 0; 0])
R = 1.0*diagm([10; 1; 1; 1])

sing = 1e-6

P = deepcopy(Q)
K_old = (R + B_ours'*P*B_ours)\B_ours'*P*A_ours
P_old = Q + A_ours'*P*(A_ours - B_ours*K_old)

K_ours = deepcopy(K_old)

for i = 1:10000
    K_new = (R + B_ours'*P_old*B_ours)\B_ours'*P_old*A_ours
    P_new = Q + A_ours'*P_old*(A_ours - B_ours*K_new)
    if norm(K_new - K_old) < 1e-5
        println("Converged at iteration $i")
        K_ours = K_new
        break
    end
    K_old = K_new
    P_old = P_new
end

################ ETH ################
Q = 1.0*diagm([0; 0; 0; 1; 1; 1; 0; 0; 0; 0.5; 0.5; 0; 0])
R = 1.0*diagm([10; 1; 1; 1])

sing = 1e-6

P = deepcopy(Q)
K_old = (R + B_eth'*P*B_eth)\B_eth'*P*A_eth
P_old = Q + A_eth'*P*(A_eth - B_eth*K_old)

K_eth = deepcopy(K_old)

for i = 1:10000
    K_new = (R + B_eth'*P_old*B_eth)\B_eth'*P_old*A_eth
    P_new = Q + A_eth'*P_old*(A_eth - B_eth*K_new)
    if norm(K_new - K_old) < 1e-5
        println("Converged at iteration $i")
        K_eth = K_new
        break
    end
    K_old = K_new
    P_old = P_new
end

"We assume that the CoM of pendulum is at the center of pendulum."

"The CoM of the pendulum is at 0.5 m."

"We assume that the CoM of pendulum is at the center of pendulum."

"The CoM of the pendulum is at 0.5 m."

"We assume that the CoM of pendulum is at the center of pendulum."

...

"We assume that the CoM of pendulum is at the center of pendulum."

"The CoM of the pendulum is at 0.5 m."

"We assume that the CoM of pendulum is at the center of pendulum."

"The CoM of the pendulum is at 0.5 m."

Converged at iteration 581
Converged at iteration 1064


In [47]:
#############################################
#################### SIM ####################
#############################################

x0 = [0; 0; 0; 1; 1; 3; 0; 0; 0; 0; 0; 0; 0]    # 13 states for ETH model

tf = 1.0
t_vec = 0:dt:tf
N = length(t_vec)
X_eth = [zeros(13) for i = 1:N] # ETH
U_bodyrate = [zeros(4) for i = 1:N-1]
X_ours = [zeros(nx) for i = 1:N] # Ours
U_torque = [zeros(nu) for i = 1:N-1]
U_eth = [zeros(4) for i = 1:N-1]
X_eth[1] = x0
X_eth[2] = x0
X_ours[1] = [0; 0; 0; 0; 0; 0; 1; 1; 3; 0; 0; 0; 0; 0; 0; 0]
X_ours[2] = [0; 0; 0; 0; 0; 0; 1; 1; 3; 0; 0; 0; 0; 0; 0; 0]

for k = 2:N-1
    _U_eth = ugoal_eth - K_eth*(X_eth[k] - xgoal_eth)
    _X_ours = observer(params, X_eth[k-1], X_eth[k])
    _U_torque = ugoal - K_ours*(_X_ours - xgoal)
    _U_bodyrate = torque_to_bodyrate(params, _U_torque, U_bodyrate[k-1], _X_ours)
    _X_eth = rk4(eth_bodyrate_dynamics, params, X_eth[k], _U_bodyrate, dt)
    X_eth[k+1] = _X_eth
    U_eth[k] = _U_eth
    X_ours[k+1] = _X_ours
    U_bodyrate[k] = _U_bodyrate
    U_torque[k] = _U_torque

    println("_X_eth: $_X_eth")
    println("_X_ours: $_X_ours")
end

Xm_eth = hcat(X_eth...)
Um_eth = hcat(U_eth...)
Um_bodyrate = hcat(U_bodyrate...)
Xm_ours = hcat(X_ours...)
Um_torques = hcat(U_torque...)

display(plot(t_vec[1:N-1],(Um_eth[2:4,1:N-1])',title = "ETH Bodyrate",
            xlabel = "time(s)", ylabel = "wx, wy, wz",
            label = ["wx" "wy" "wz"]))
display(plot(t_vec[1:N-1],(Um_bodyrate[2:4,1:N-1])',title = "Ours Bodyrate",
            xlabel = "time(s)", ylabel = "wx, wy, wz",
            label = ["wx" "wy" "wz"]))

display(plot(t_vec[1:N-1],(Xm_ours[1:3,1:N-1])',title = "Ours Attitude",
            xlabel = "time(s)", ylabel = "γ, β, α",
            label = ["γ" "β" "α"]))
display(plot(t_vec[1:N-1],(Xm_ours[4:6,1:N-1])',title = "Ours Ang. Vel.",
            xlabel = "time(s)", ylabel = "γ_dot, β_dot, α_dot",
            label = ["γ_dot" "β_dot" "α_dot"]))
display(plot(t_vec[1:N-1],(Um_torques[2:4,1:N-1])',title = "Ours Torques",
            xlabel = "time(s)", ylabel = "τx, τy, τz",
            label = ["τx" "τy" "τz"]))

display(plot(t_vec[1:N-1],(Xm_eth[4:5,1:N-1])',title = "ETH Position",
            xlabel = "time(s)", ylabel = "x, y, z",
            label = ["x" "y" "z"]))
display(plot(t_vec[1:N-1],(Xm_ours[7:8,1:N-1])',title = "Ours Position",
            xlabel = "time(s)", ylabel = "x, y, z",
            label = ["x" "y" "z"]))
display(plot(t_vec[1:N-1],(Xm_eth[10:11,1:N-1])',title = "ETH Pend. Pos.",
            xlabel = "time(s)", ylabel = "r, s",
            label = ["r" "s"]))
display(plot(t_vec[1:N-1],(Xm_ours[13:14,1:N-1])',title = "Ours Pend. Pos.",
            xlabel = "time(s)", ylabel = "theta, phi",
            label = ["theta" "phi"]))      
display(plot(t_vec[1:N-1],(Xm_eth[12:13,1:N-1])',title = "ETH Pend. Vel.",
            xlabel = "time(s)", ylabel = "r_dot, s_dot",
            label = ["r_dot" "s_dot"]))
display(plot(t_vec[1:N-1],(Xm_ours[15:16,1:N-1])',title = "Ours Pend. Vel.",
            xlabel = "time(s)", ylabel = "theta_dot, phi_dot",
            label = ["theta_dot" "phi_dot"]))          

# Xm = hcat(X...)
# Um = hcat([ugoal - K_ours*(X[k] - xgoal) for k = 1:N]...)
# display(plot(t_vec[1:N-1],(Xm[4:5,1:N-1])',title = "Position",
#             xlabel = "time(s)", ylabel = "x, y, z",
#             label = ["x" "y" "z"]))

# display(plot(t_vec[1:N-1],(Xm[10:11,1:N-1])',title = "Pend. Pos.",
#             xlabel = "time(s)", ylabel = "theta, phi",
#             label = ["theta" "phi"]))        
# display(plot(t_vec[1:N-1],(Xm[12:13,1:N-1])',title = "Pend. Vel.",
#             xlabel = "time(s)", ylabel = "theta_dot, phi_dot",
#             label = ["theta_dot" "phi_dot"]))

_X_eth: [-0.0028276709577896944, 0.0031140336819209915, -2.258545490489474e-5, 1.0000005094754105, 1.000000461958202, 2.999999999276802, 0.0001528102740446535, 0.00013862288312555118, -2.8927886298824034e-7, -5.09475410562287e-7, -4.619582019080472e-7, -0.00015283527956004515, -0.00013864552435023843]
_X_ours: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 3.0, 0.0, 0.0, 0.0, 0.0, -0.0, 0.0, -0.0]
_X_eth: [-0.008139082482553886, 0.008914058402369487, -8.856933829517143e-5, 1.0000045145282632, 1.0000041021847288, 2.9999999841194214, 0.0007430348575469131, 0.0006762510616181402, -3.788669238096176e-6, -4.516027668519373e-6, -4.103544497104602e-6, -0.0007434563157444636, -0.000676633558090974]
_X_ours: [-0.0028276709577896944, 0.0031140336819209915, -2.258545490489474e-5, -0.2827670957789694, 0.31140336819209913, -0.002258545490489474, 1.0000005094754105, 1.000000461958202, 2.999999999276802, 0.0001528102740446535, 0.00013862288312555118, -2.8927886298824034e-7, -1.0189508211251855e-6, 9.239164

DomainError: DomainError with -7.751563704636033e-5:
sqrt will only return a complex result if called with a complex argument. Try sqrt(Complex(x)).

In [26]:
Xm_ours[1:16, 1:10]

16×10 Matrix{Float64}:
  0.0   0.0   0.0  -0.0286781    -0.0794185    …  -0.391276    -0.467446
  0.0   0.0   0.0   0.0307733     0.0810577        0.324853     0.380849
  0.0   0.0   0.0  -0.000627275  -0.00384867      -0.0737018   -0.102551
  0.0   0.0   0.0  -2.86781      -5.07404         -8.27112     -7.61701
  0.0   0.0   0.0   3.07733       5.02843          5.88776      5.59958
  0.0   0.0   0.0  -0.0627275    -0.322139     …  -2.6149      -2.88497
  1.0   1.0   1.0   1.00001       1.00004          1.0016       1.00264
  1.0   1.0   1.0   1.0           1.00004          1.00165      1.00275
  3.0   3.0   3.0   3.0           3.0              2.99973      2.99946
  0.0   0.0   0.0   0.00151621    0.00702134       0.0867932    0.121159
  0.0   0.0   0.0   0.00139872    0.00666332   …   0.092265     0.130244
  0.0   0.0   0.0  -2.8925e-5    -0.000345771     -0.0207232   -0.0350049
  0.0   0.0   0.0  -1.01307e-5   -8.73221e-5      -0.00321101  -0.00529312
 -0.0  -0.0  -0.0   9.30179e-6 

In [8]:
using Printf

In [9]:
function print_np_matrix(mat)
    rows, cols = size(mat)
    for i in 1:rows
        print("[")
        for j in 1:cols
            @printf("%10.6g, ", mat[i, j])
        end
        print("]")
        println()
    end
end
function print_latex_matrix(mat)
    rows, cols = size(mat)
    for i in 1:rows
        for j in 1:cols
            @printf("%10.5g ", mat[i, j])
            print("&")
        end
        println()
    end
end

print_latex_matrix (generic function with 1 method)

In [15]:
# print_np_matrix(A_ours)
print_np_matrix(B_ours)

[         0,  0.0156079, -0.000629628, -0.000146218, ]
[         0, -0.000629628,   0.017349, -0.00021998, ]
[         0, -0.000146218, -0.00021998,  0.0101024, ]
[         0,    3.12159,  -0.125926, -0.0292435, ]
[         0,  -0.125926,    3.46981, -0.0439959, ]
[         0, -0.0292435, -0.0439959,    2.02049, ]
[         0, -5.14721e-08, 1.41828e-06, -1.79833e-08, ]
[         0, -1.27595e-06, 5.14721e-08, 1.19533e-08, ]
[     5e-05,          0,          0,          0, ]
[         0, -2.05888e-05, 0.000567314, -7.19333e-06, ]
[         0, -0.00051038, 2.05888e-05, 4.78132e-06, ]
[      0.01,          0,          0,          0, ]
[         0, 7.72082e-08, -2.12743e-06, 2.6975e-08, ]
[         0, -1.91392e-06, 7.72082e-08,  1.793e-08, ]
[         0, 3.08833e-05, -0.000850971,  1.079e-05, ]
[         0, -0.000765569, 3.08833e-05, 7.17198e-06, ]


In [21]:
print_np_matrix(A_eth)
print_np_matrix(B_eth)

[         1,          0,          0,          0,          0,          0,          0,          0,          0,          0,          0,          0,          0, ]
[         0,          1,          0,          0,          0,          0,          0,          0,          0,          0,          0,          0,          0, ]
[         0,          0,          1,          0,          0,          0,          0,          0,          0,          0,          0,          0,          0, ]
[         0,  0.0004905,          0,          1,          0,          0,       0.01,          0,          0,          0,          0,          0,          0, ]
[-0.0004905,          0,          0,          0,          1,          0,          0,       0.01,          0,          0,          0,          0,          0, ]
[         0,          0,          0,          0,          0,          1,          0,          0,       0.01,          0,          0,          0,          0, ]
[         0,     0.0981,          0,          